In [1]:
import numpy as np 
import pandas as pd
import os
import glob
import re
import sys

import plotnine
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import folium
from folium import plugins
import json

from branca.element import CssLink, Figure, JavascriptLink, MacroElement
from jinja2 import Template

In [2]:
files = glob.glob('dataset/*.csv')
data = []
for i in files :
    i = re.sub("dataset/", "", i)
    data.append(re.sub(".csv","",i))
for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])

### 직선에 색을 넣고자 한다. patientinfo 데이터에서 contact number 에 따라 색을 구분하기로 했다.

In [41]:
contnum = PatientInfo.iloc[:,[0,12]]

In [42]:
contnum.describe()

,patient_id,contact_number
count,2.218000e+03,411.000000
mean,4.014678e+09,24.128954
std,2.192419e+09,91.087792
min,1.000000e+09,0.000000
25%,1.700000e+09,2.000000
50%,6.001000e+09,5.000000
75%,6.004000e+09,16.000000
max,7.000000e+09,1160.000000


#### 0 일 경우 초록색, 0 초과 10 이하일 경우 파란색, 10 초과 100 이하일 경우 노란색, 100 초과일 경우 빨간색으로 하자. 제일 많은 사람은 검정색.

In [43]:
clr = []
for i in range(len(contnum)) :
    if contnum.contact_number[i] == 0 :
        clr.append("green")
    elif contnum.contact_number[i] <= 10 :
        clr.append("blue")
    elif contnum.contact_number[i] <= 100 :
        clr.append("yellow")
    elif contnum.contact_number[i] == max(contnum.contact_number) :
        clr.append("black")
    else : 
        clr.append("red")

In [44]:
contnum["color"] = clr

/Users/seonwoolim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
wcontnum = pd.merge(PatientRoute,contnum,
        how ='left',
        on = 'patient_id').fillna(0)

In [46]:
route_geo = gpd.GeoDataFrame(wcontnum,geometry=gpd.points_from_xy(PatientRoute.longitude,PatientRoute.latitude))

### 다른 환자들 간에는 직선이 이어져서는 안된다.

In [47]:
linerange = route_geo[route_geo['patient_id'] == route_geo['patient_id'].shift(1)].index
linerange = linerange.drop(linerange[-1])

In [48]:
lines = [ {
                "coordinates": [
                 [route_geo["longitude"][i] , route_geo["latitude"][i] ],
                 [route_geo["longitude"][i+1] , route_geo["latitude"][i+1],]
                   ],
                "dates": [
                 route_geo["date"][i],
                 route_geo["date"][i+1]
                   ],
                "color": route_geo["color"][i]
                     } for i in linerange ] 
features = [
     {
         'type': 'Feature',
         'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
                },
         'properties': {
            'times': line['dates'],
            'style': {
                'color' : line['color']
            },
            'icon' : 'circle',
            'iconstyle':{
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 3
                }
                }
        }
        for line in lines
    ]

In [51]:
m = folium.Map(
  location=[36.5053542,127.7043419],
  zoom_start=8
)
plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT6H', add_last_point=True).add_to(m)
m.save('with_color.html')
m